In [2]:
import numpy as np
import dicom
import os
import cv2
import tensorflow as tf

In [17]:
'''
  Gets 3d data from DICOM file as (depth, width, height)
'''
def get_3d_data(path):
  slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
  slices.sort(key=lambda x: int(x.InstanceNumber))
  return np.stack([s.pixel_array for s in slices])


In [18]:
'''
  Makes one CT scan as a batch of 'image's with shape (224,224,3)
  
'''
def make_batch(data_3d):
  data_3d[data_3d == -2000] = 0

  batch = []
  for i in range(0, data_3d.shape[0] - 2):
    tmp = []
    for j in range(3):
      img = data_3d[i + j]
      img = 255.0 / np.amax(img) * img
      img = cv2.equalizeHist(img.astype(np.uint8))
      img = cv2.resize(img, (224, 224))
      tmp.append(img)

    # transpose 3,224,224 to 224,224,3 for the vgg16 feed dict.
    tmp = np.transpose(tmp, (1, 2, 0))
    batch.append(np.array(tmp))

  batch = np.array(batch)
  return batch


In [ ]:
'''
  Gets the graph definition from the pre-trained vgg16-model.
  
'''
def get_graph_def():
  with open("vgg16.tfmodel", mode='rb') as f:
    file_content = f.read()

  graph_def = tf.GraphDef()
  graph_def.ParseFromString(fileContent)

  return graph_def


In [ ]:
'''
  Extracts features from the CT scans and save them locally.

'''
def extract_features(data):
  graph_def = get_graph_def()
  print('graph loaded from disk')

  images = tf.placeholder("float", [None, 224, 224, 3])
  tf.import_graph_def(graph_def, input_map={"images": images})

  graph = tf.get_default_graph()

  with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)

    #####################
    # TODO: should be in a loop
    data = get_3d_data('../data/sample/images/0a0c32c9e08cc2ea76a71649de56be6d')
    batch = make_batch(data)
    feed_dict = {images: batch}

    fc_tensor = graph.get_tensor_by_name("import/fc8:0")
    feats = sess.run(fc_tensor, feed_dict=feed_dict)
    # TODO: save the feats, do we need average or other operations to have a 
    # smaller feature vector over the batch?

  return feats


In [ ]:
'''
  Trains the model with extracted feature.

'''
def train(data):
  extractor = get_extractor()

  # feats = get_features(extractor, data)


  return model


In [ ]:
'''
  Makes prediction with the model trained.
  
'''
def predict(model, x):
  return preds


In [20]:
if __name__ == '__main__':
#   data = get_3d_data('../data/sample/images/0a0c32c9e08cc2ea76a71649de56be6d')
#   batch = make_batch(data)
#   print(batch.shape)
  
   # model = train(train_data)
   # preds = predict(model, test_data)

(131, 224, 224, 3)
